# pipeline all models 

In [1]:
import numpy as np
import cv2
import sklearn
import pickle

In [6]:
# face detection
face_detector_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt'
                                               ,'./models/res10_300x300_ssd_iter_140000.caffemodel')
# # feature extraction
face_feature_model = cv2.dnn.readNetFromTorch('./models/openface.nn4.small2.v1.t7')
# # face recognition 
face_recogniation_model = pickle.load(open('./models/machinelearning_face_person_identity.pkl',mode ='rb'))
# # emotion recogniation model
emotion_recognition_model = pickle.load(open('./models/machinelearning_face_emotion.pkl',mode ='rb'))

In [9]:
# pipeline model
img = cv2.imread('./data/test_images.jpg')
image = img.copy()
h,w = img.shape[:2]
# face detection 
img_blob = cv2.dnn.blobFromImage(img,1,(300,300),(104,177,123),swapRB= False,crop=False)
face_detector_model.setInput(img_blob)
detections = face_detector_model.forward()

if len(detections) > 0:
    for i , confidence in enumerate(detections[0,0,:,2]):
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            startx,starty,endx,endy = box.astype(int)
            
            cv2.rectangle(image,(startx,starty),(endx,endy),(0,255,0))


cv2.imshow('pipeline',img)
cv2.waitKey(0)
cv2.destroyAllWindows()